# BERT 문장 임베딩 모델을 이용한 챗봇

In [1]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=30e09b1a40bbb418bfd6cdafa832100d7ad8f926f2154fc1a619e29f315da460
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 라이브러리 임포트

In [3]:
import pandas as pd
import numpy as np

from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer

## 데이터 전처리

### 사용할 데이터 불러오기

In [57]:
# 사용할 데이터 불러오기(https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86)
train_data = pd.read_excel('/content/drive/MyDrive/AI/7.NaturalLanguageProcessing/data/chatbot_Training.xlsx')
# 데이터 확인
train_data

,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN
...,...,...,...,...,...,...
51628,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,가족과의 문제 때문에 속상하시겠어요.,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.,지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.
51629,남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져.,대인관계에 대한 어려움 때문에 걱정되시고 속상하시겠어요.,사람들을 만나는 것이 어려워. 자꾸 사람들을 의심하게만 되고 말이야.,어떻게 하면 지금의 상황에 변화를 만들어낼 수 있을까요?,사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지...,원하시는 대로 가지고 계시던 걱정이 잘 해결되셨으면 좋겠어요.
51630,안녕.,안녕하세요. 오늘도 좋은 하루 보내고 계신가요?,넌 이름이 뭐니?,제 이름은 위로봇이에요.,너 정말 똑똑하구나?,고마워요.
51631,좋은 아침이야.,어서오세요. 좋은 아침이에요.,점심 메뉴 추천 해줘,아쉽지만 당신의 취향을 몰라 추천해드리기가 어렵겠네요.,좋은 밤 보내,안녕히 주무세요.


### 데이터 Shape 변환

In [85]:
def set_data_shape(train_data):
  train_data1 = train_data[['사람문장1', '시스템문장1']]
  train_data2 = train_data[['사람문장2', '시스템문장2']]
  train_data3 = train_data[['사람문장3', '시스템문장3']]
  data_list = [train_data1, train_data2, train_data3]

  for i in range(len(data_list)):
    data_list[i] = data_list[i].rename(columns={f"사람문장{i+1}":"Q", f"시스템문장{i+1}":"A"})

  set_data = pd.concat(data_list)

  return set_data

In [86]:
set_data_shape(train_data)

,Q,A
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
...,...,...
51628,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.
51629,사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지...,원하시는 대로 가지고 계시던 걱정이 잘 해결되셨으면 좋겠어요.
51630,너 정말 똑똑하구나?,고마워요.
51631,좋은 밤 보내,안녕히 주무세요.


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154899 entries, 0 to 51632
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Q       145964 non-null  object
 1   A       145964 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


### 결측치 처리

In [ ]:
train_data.isnull().sum()

Q    8935
A    8935
dtype: int64

In [ ]:
train_data = train_data.dropna()
train_data

,Q,A
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
...,...,...
51628,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.
51629,사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지...,원하시는 대로 가지고 계시던 걱정이 잘 해결되셨으면 좋겠어요.
51630,너 정말 똑똑하구나?,고마워요.
51631,좋은 밤 보내,안녕히 주무세요.


In [ ]:
train_data.isnull().sum()

Q    0
A    0
dtype: int64

### 인덱스 재배열

In [ ]:
train_data = train_data.reset_index().drop(['index'], axis=1)
train_data

,Q,A
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
...,...,...
145958,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.
145959,사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지...,원하시는 대로 가지고 계시던 걱정이 잘 해결되셨으면 좋겠어요.
145960,너 정말 똑똑하구나?,고마워요.
145961,좋은 밤 보내,안녕히 주무세요.


### 정규표현식 적용

In [ ]:
train_data['Q'] = train_data['Q'].str.replace("[^가-힣?.!,]", " ")
train_data['Q'] = train_data['Q'].str.replace('^ +', "")
train_data

<ipython-input-15-866a6217a2b1>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['Q'] = train_data['Q'].str.replace("[^가-힣?.!,]", " ")
<ipython-input-15-866a6217a2b1>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['Q'] = train_data['Q'].str.replace('^ +', "")


,Q,A
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
...,...,...
145958,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.
145959,사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지...,원하시는 대로 가지고 계시던 걱정이 잘 해결되셨으면 좋겠어요.
145960,너 정말 똑똑하구나?,고마워요.
145961,좋은 밤 보내,안녕히 주무세요.


In [ ]:
train_data['A'] = train_data['A'].str.replace("[^가-힣?.!,]", " ")
train_data['A'] = train_data['A'].str.replace('^ +', "")
train_data

<ipython-input-16-b21dd27aa159>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['A'] = train_data['A'].str.replace("[^가-힣?.!,]", " ")
<ipython-input-16-b21dd27aa159>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['A'] = train_data['A'].str.replace('^ +', "")


,Q,A
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
...,...,...
145958,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.
145959,사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지...,원하시는 대로 가지고 계시던 걱정이 잘 해결되셨으면 좋겠어요.
145960,너 정말 똑똑하구나?,고마워요.
145961,좋은 밤 보내,안녕히 주무세요.


### 처리후 데이터 저장

In [ ]:
train_data.to_csv(f"/content/drive/MyDrive/AI/7.NaturalLanguageProcessing/data/preprocessing_data.csv",encoding='utf-8', index=False)

## 사용할 모델 불러오기

In [ ]:
# 사용할 모델 불러오기
# https://huggingface.co/models?library=sentence-transformers
# 위 링크에서 다양한 문서 임베딩 모델들을 불러와서 사용할 수 있다.
# 사용한 모델들:
# sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens
# jhgan/ko-sroberta-multitask

model = SentenceTransformer('ddobokki/klue-roberta-base-nli-sts-ko-en')

### Q(질문)문장 임베딩값 데이터 프레임에 업데이트

In [ ]:
# Q의 문장 임베딩값 따로 저장

train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [ ]:
train_data.head()

,Q,A,embedding
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,"[-0.812368, 0.38922358, 0.18056267, 0.09498362..."
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,"[0.693432, -0.40477628, 0.18209185, -0.5524977..."
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,"[0.19509983, -0.123300076, -0.011882742, 0.317..."
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,"[-0.06959631, -0.022704173, 0.21452557, -0.208..."
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,"[-0.2085296, -0.79528505, 0.12313598, 0.296531..."


### 코사인 유사도 구하는 함수

In [ ]:
# 두 벡터에서 코사인 유사도 구하는 함수 정의

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

### 유사도가 비슷한 A(대답)문장을 구해주는 함수

In [ ]:
# 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과
# 챗봇 데이터의 임베딩 열(train_data['embedding'])에 저장해둔
# 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여
# 코사인 유사도 값이 가장 높은 질문 샘플을 찾아내기.
# 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴

def return_similar_answer(input):
    embedding = model.encode(input)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

## 챗봇 구현

In [ ]:
start = input('위로봇을 시작 하시려면 "안녕"을 입력하세요\n')
print(return_similar_answer(start))

while start == "안녕":

  input_text = input('\n위로봇에게 하고 싶은 말을 입력하세요, 종료를 원하시면 "종료"를 입력하세요\n')

  if input_text == "종료":
    print("\n위로봇을 종료합니다.")
    break

  else:
    print(return_similar_answer(input_text))

위로봇을 시작 하시려면 "안녕"을 입력하세요
안녕
안녕하세요. 오늘도 좋은 하루 보내고 계신가요?

위로봇에게 하고 싶은 말을 입력하세요, 종료를 원하시면 "종료"를 입력하세요
내가 좋은 회사에 취직할 수 있을까?
힘내세요. 무엇을 하면 취업에 도움이 될 수 있을까요?

위로봇에게 하고 싶은 말을 입력하세요, 종료를 원하시면 "종료"를 입력하세요
공부를 열심히 해야지
공부 열심히 해서 좋은 결실 있길 바랄게요.

위로봇에게 하고 싶은 말을 입력하세요, 종료를 원하시면 "종료"를 입력하세요
위로가 되네 고마워
다른 고민이 생기면 또 찾아와주세요

위로봇에게 하고 싶은 말을 입력하세요, 종료를 원하시면 "종료"를 입력하세요
종료

위로봇을 종료합니다.


In [ ]:
return_similar_answer('오늘 날씨는 어때?')                                  # 트랜스포머 모델의 대답 : 날씨가 좋군요 .

'어젠 정말 화창했는데 오늘은 좀 흐리네요! 기분이 울적해지셨나 봐요.'

In [ ]:
return_similar_answer('놀러가고 싶다')                                      # 트랜스포머 모델의 대답 : 즐거운 학교생활을 보내길 바라요 .

'성적이 잘 나와서 놀러 갈 수 있는 상황이시군요.'

In [ ]:
return_similar_answer('너는 이름이 뭐야?')                                 # 트랜스포머 모델의 대답 : 이름을 불러주는 방법이 있어서 기분이 안 좋으시겠어요

'제 이름은 위로봇이에요.'

In [ ]:
return_similar_answer('저녁 뭐 먹을지 고민돼')                              # 트랜스포머 모델의 대답 : 저녁 먹을 때보다는 모임에 참는게 고민되시는군요

'대화로 남편 분의 어려운 상황이 잘 풀리길 바라요.'

In [ ]:
return_similar_answer('나 너무 힘들어')                                     # 트랜스포머 모델의 대답 : 지금 상황에서 할 수 있는 게 어떤 것이 있을까요 ?

'힘드신 이유가 무엇 때문일까요?'

In [ ]:
return_similar_answer('머리가 복잡해')                                      # 트랜스포머 모델의 대답 : 복잡한 상황이군요 . 이 문제에 대해 상담할 분이 있을까요 ?

'이번 발표 준비는 어떻게 하실 생각이세요?'

In [ ]:
return_similar_answer('행복해')                                             # 트랜스포머 모델의 대답 : 행복한 학교생활을 응원할게요 .

'그분들과 좋은 인연 이어가면 좋겠어요.'

In [ ]:
return_similar_answer('재밌는 이야기 해줘')                                 # 트랜스포머 모델의 대답 : 재밌는 친구를 만나셔서 좋으시겠어요 .

'고등학교 친구들을 만나 정말 행복하셨군요.'

In [ ]:
return_similar_answer('영화 보러 갈래?')                                    # 트랜스포머 모델의 대답 : 영화 보는 방법을 생각하고 계시는군요 .

'의미 있는 시간을 갖게 되길 바라요.'

In [ ]:
return_similar_answer('질문 좀 그만해')                                     # 트랜스포머 모델의 대답 : 질문이 적해서 적당한 질문을 가 없으시군요 .

'남편분이 그만 물어봤으면 하는군요.'

In [ ]:
return_similar_answer('귀찮아')                                             # 트랜스포머 모델의 대답 : 귀찮다고 생각하시는군요 .

'어떤 일이 귀찮으신건지 더 자세히 말씀해 주시겠어요?'

In [ ]:
return_similar_answer('일은 왜 해도 해도 끝이 없을까? 화가 난다.')          # 트랜스포머 모델의 대답 : 많이 힘드실 것 같아요 . 지금 이 기분을 벗어나기 위해 할 수 있는 일이 있을까요 ?

'많이 힘드시겠어요. 주위에 의논할 상대가 있나요?'

> * 데이터 자체가 감성분석 대화 데이터라 감정적인 방향의 질문의 대답만 정확도가 높습니다.
> * 직접 모델을 구현하여 학습시킨 생성형 챗봇처럼 대답을 직접 만들어내서 하지는 못 하지만 질문이 학습시킨 내용에서 크게 벗어나지 않는다면 도출해내는 대답이 생성형 챗봇보다 좋은 경우도 있습니다.
> * 위의 결과로 봤을때 특정분야에 대한 데이터로 실행을 할 경우 생성형 챗봇보다 구현과 데이터 처리는 간단하지만 대답은 크게 다르지 않아 사용하기 간편할거 같습니다만 더 다양한 질문에 대처하기 위해서는 더 다양하고 폭넓은 범위의 데이터 셋이 필요 할 것 같습니다.
> * 트랜스포머 모델과 비교했을 때 문법적으로 말이 안되는 대답을 하는 경우가 없습니다.